<a href="https://colab.research.google.com/github/tibayo1/Leukemia-DL-Diagnosis/blob/master/leukemia_dl_diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
Journal: https://www.biorxiv.org/content/10.1101/564039v1.full.pdf
Dataset: https://www.kaggle.com/rashasalim/blood-smear-images-for-aml-diagnosis
